In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import os # for handling the directory
import shutil
import json
import tqdm

In [32]:
train_json = '/hhome/ps2g07/document_analysis/github/Project_Synthesis2-/Datasets/Synthetics_DS/train.json'
with open(train_json) as f:
    train_data = json.load(f)

val_json = '/hhome/ps2g07/document_analysis/github/Project_Synthesis2-/Datasets/Synthetics_DS/val.json'
with open(val_json) as f:
    val_data = json.load(f)

In [103]:
# Create an auxiliary dictionary to map image_id to filename
def obtain_new_dict(data):
    id_to_filename = {image['id']: {'file_name': image['file_name'], 'width': image['width'], 'height': image['height']} for image in data['images']}

    # Initialize your dictionary with filenames as keys
    new_dict = {values['file_name']: {'width': values['width'], 'height': values['height'], 'images': []} for values in id_to_filename.values()}

    # Iterate through annotations a single time, adding them to the corresponding list in new_dict
    for annotation in data['annotations']:
        image_id = annotation['image_id']
        if image_id in id_to_filename:  # Check if the image_id exists in the mapping
            filename = id_to_filename[image_id]['file_name']
            new_dict[filename]['images'].append(annotation)
    return new_dict


new_dict_train = obtain_new_dict(train_data)
new_dict_val = obtain_new_dict(val_data)

In [116]:
def create_txt_files(data, folder_path):
    for file_name in data.keys():
        with open(f'{folder_path}{file_name[:-4]}.txt', 'w') as f:
            for annotation in data[file_name]['images']:
                x, y, w, h = annotation['bbox']
                x_center = x + w/2
                y_center = y + h/2
                x_center /= data[file_name]['width']
                y_center /= data[file_name]['height']
                w /= data[file_name]['width']
                h /= data[file_name]['height']
                class_id = annotation['category_id'] - 1
                f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

# Create the txt files for the train and val datasets
create_txt_files(new_dict_train, '/hhome/ps2g07/document_analysis/github/Project_Synthesis2-/Nil/yolo_dir/data/labels/train/')
create_txt_files(new_dict_val, '/hhome/ps2g07/document_analysis/github/Project_Synthesis2-/Nil/yolo_dir/data/labels/val/')

In [ ]:
[{'supercategory': '', 'id': 1, 'name': 'text'},
 {'supercategory': '', 'id': 2, 'name': 'title'},
 {'supercategory': '', 'id': 3, 'name': 'list'},
 {'supercategory': '', 'id': 4, 'name': 'table'},
 {'supercategory': '', 'id': 5, 'name': 'figure'},
 {'supercategory': '', 'id': 6, 'name': 'signature'},
 {'supercategory': '', 'id': 7, 'name': 'stamp'},
 {'supercategory': '', 'id': 8, 'name': 'qr'},
 {'supercategory': '', 'id': 9, 'name': 'barcode'}]

In [1]:
from ultralytics import YOLO


model = YOLO()

model.train??

In [ ]:
[34m[1mengine/trainer: [0mtask=detect, mode=train, model=dla-model.pt, data=yolo_config.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, nms=False, lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, dfl=1.5, pose=12.0, kobj=1.0, label_smoothing=0.0, nbs=64, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, bgr=0.0, mosaic=1.0, mixup=0.0, copy_paste=0.0, auto_augment=randaugment, erasing=0.4, crop_fraction=1.0, cfg=None, tracker=botsort.yaml, save_dir=/hhome/ps2g07/runs/detect/train17
